# Regulatory Genomics Lecture Exercise

## Install the dependencies

In addition to the pre-installed packages like numpy, pandas, matplotlib, keras, tensorflow, we'll install concise, a keras extension for regulatory genomics developed in the Gagneur lab: https://github.com/gagneurlab/concise.

In [ ]:
! pip install concise

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import concise
from concise.preprocessing import encodeDNA
from concise.utils import PWM

from concise.utils.plot import seqlogo, seqlogo_fig

In [ ]:
# Used additional packages
%matplotlib inline

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

## Load simulated data

## Get the data

We are going to use simulated data of 10,000 500 bp long sequences with the positive set containing an instance of the TAL1 motif:
![TAL1 known4](http://compbio.mit.edu/encode-motifs/logos/table/logos/small/rev/TAL1_known4.png)
and the negative set will be random sequences. The data were simulated using simDNA https://github.com/kundajelab/simdna by Johnny Israeli and were deposited to https://github.com/kundajelab/dragonn.

In [ ]:
!wget 'https://github.com/kundajelab/dragonn/raw/master/paper_supplement/simulation_data/GC_fraction0.4motif_nameTAL1_known4num_neg10000num_pos10000seq_length500.npz'

We now list all files in our directory

In [ ]:
ls

## Load the data
The following function can load the data.

In [ ]:
import numpy as np

def load_simulated_data(path):
    """Load the simulated dataset
    
    Args:
      path: path to the .npz file c
    """
    data = np.load(path)

    x_train = data["X_train"].squeeze(1).swapaxes(1,2)
    x_valid = data["X_valid"].squeeze(1).swapaxes(1,2)
    y_train = data['y_train']
    y_valid = data['y_valid']
    return (x_train, y_train), (x_valid, y_valid)

In [ ]:
(x_train, y_train), (x_test, y_test) = load_simulated_data("GC_fraction0.4motif_nameTAL1_known4num_neg10000num_pos10000seq_length500.npz")

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_test.shape

In [ ]:
y_test.shape

As you can see, there are 12.8k training examples and 3.2k test examples.

### Visualize the first sequence

The response value is a binary variable:

In [ ]:
y_train[:5]

In [ ]:
y_train.mean()

In [ ]:
y_test.mean()


As the means are close to 0.5, there are balanced classes, i.e. roughly the same number of positive and negative instances. This ratio is the same for training and testing. This is an ideal situation for training a classifier.


The input is the one-hot-encoded DNA sequence:

In [ ]:
x_train[0][:10]

In [ ]:
from concise.preprocessing.sequence import one_hot2string, DNA

In [ ]:
one_hot2string(x_train[:1], DNA)[0]

## Load TAL1 motif Position-specific Weight Matrix (PWM)

Get PWM of TAL1 motif

In [ ]:
! wget https://github.com/gagneurlab/SystemGeneticsDL/raw/master/TAL1_known4.npy

In [ ]:
pwm = np.load("TAL1_known4.npy")

In [ ]:
seqlogo_fig(pwm);

## Predict binding with pwm scan model

In this section, you will implement a PWM scan model, which is a convolution operation

In [ ]:
# A naiv version
def pwm_scan(sequence, pwm, pad=0, stride=1):
    """ sequence: (N, L, 4)
    output length given by L' = 1 + (L + 2*P - F) / stride
    """
    pwm = np.log(pwm) # transform frequencies into log scale to have an additive model
    assert len(sequence.shape) == 3
    assert pwm.shape[1] == 4
    N, L, _ = sequence.shape
    F, _ = pwm.shape
    S = stride
    assert (L + 2 * pad - F) % S == 0, "Size not fit."
    L_out = int(1 + (L + 2 * pad - F) / S)
    out = np.zeros((N, L_out))

    x_pad = np.pad(sequence, ((0, 0), (1, 1), (0,0)), mode='constant')

    ###################################################################
    # Fill your code here
    # Write the PWM scan (convolution) operation, store the output
    # into `out` array.
    
    
    ###################################################################


    return out

In [ ]:
scores = pwm_scan(x_test, pwm)

In [ ]:
scores.shape

## Summarize PWM scan ouput per sequence with two pooling strategies

Try two pooling strategies:
  * GloabalAveragePooling: Take average score per sequence 
  * GloabalMaxPooling: take max activation score

### Averange Pooling

In [ ]:
mean_scores = ## Fill your code here

In [ ]:
plt.boxplot([mean_scores[y_test.flatten()], mean_scores[~y_test.flatten()]], labels=['Binding','Not Binding'])
plt.ylabel("PWM mean score")
plt.show()

### Max Pooling

In [ ]:
max_scores = ## Fill your code here

In [ ]:
plt.boxplot([max_scores[y_test.flatten()], max_scores[~y_test.flatten()]], labels=['Binding','Not Binding'])
plt.ylabel("PWM mean score")
plt.show()

### Check performance

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
roc = {}
roc['mean_roc'] = roc_auc_score(y_test, mean_scores)
roc['max_roc'] = roc_auc_score(y_test, max_scores)

In [ ]:
fpr = {}
tpr = {}
fpr['Mean'], tpr["Mean"], _ = roc_curve(y_test, mean_scores)
fpr['Max'], tpr["Max"], _ = roc_curve(y_test, max_scores)

In [ ]:
plt.plot(fpr['Mean'], tpr["Mean"], color='darkorange', 
        label = 'AveragePooling ROC={%0.2f}' % roc['mean_roc'])
plt.plot(fpr['Max'], tpr["Max"], color='cornflowerblue', 
        label = 'MaxPooling ROC={%0.2f}' % roc['max_roc'])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

## Build a convolutional neural network

The following code will give you an end-to-end example for training, evaluating and visualizing convolutional neural networks for DNA sequence models.

### Define the model

We'll use Keras (https://keras.io), a popular deep learning library wrapping frameworks like TensorFlow to define and train a neural network. After running the code, go to the documentation and read more about Keras.


https://keras.io/getting-started/sequential-model-guide/

We will now implement a neural network (sketch below) consisting of one convolutional filter whose outputs are  transfomed by the ReLU activation function ($\text{ReLU}(x) = \max(0,x)$). On the resulting vector, we keep the maximal value ("max pooling") giving us a 1-long vector. We then perform a linear transformation of this 1-long vector into a single scalar ("Dense layer") which we map to the $[0,1]$ interval with sigmoid activation function $\text{sigm}(x) = \frac{1}{1+\exp(-x)}$. The interpratation of the output is the probability that the sequence is bound by the transcription factor TAL1.

In [ ]:
## required keras modules
from keras.models import Model, load_model, Sequential
import keras.layers as kl
import keras.optimizers as ko

For the convolution operation, we will use  `ConvDNA` from `concise` package. It offers convenient functions to visualize the learned convolution filters.

In [ ]:
import concise.layers as cl

Model achitecture: ConvDNA (which is Conv1D) --> GlobalMaxPooling1D  -->  Dense --> Sigmoid

In [ ]:
model = Sequential()
###################################################################
# Fill your code here
# Use model.add() to add layers
# ConvDNA: cl.ConvDNA()
# GlobalMaxPooling1D: kl.GlobalMaxPooling1D()
# Dense: kl.Dense()
# Sigmoid: kl.Activation('sigmoid')
###################################################################

In [ ]:
model.summary()

The following image shows the initial weights:

In [ ]:
model.layers[0].plot_weights(figsize=(20, 3), plot_type="motif_pwm_info");

### Train the model

In [ ]:
from keras import optimizers

model.compile(optimizer=optimizers.Adam(lr=0.05), loss="binary_crossentropy")

from keras.callbacks import EarlyStopping

model.fit(x=x_train, y=y_train, epochs=300, verbose=2,
          callbacks=[
              EarlyStopping(patience=5)
          ],
          validation_split=.2
         )

In [ ]:
model.layers[0]

In [ ]:
model.layers[0].plot_weights(figsize=(20, 3), plot_type="motif_pwm_info");

In [ ]:
y_test_pred = model.predict(x_test)

roc['ConvNet1Motif'] = roc_auc_score(y_test, y_test_pred)

In [ ]:
fpr['ConvNet1Motif'], tpr["ConvNet1Motif"], _ = roc_curve(y_test, y_test_pred)

In [ ]:
plt.plot(fpr['ConvNet1Motif'], tpr["ConvNet1Motif"], color='darkorange', 
        label = 'ConvNet1Motif ROC={%0.2f}' % roc['ConvNet1Motif'])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

### Build a model with 4 filters

We will now implement a neural network (sketch below) consisting of 4 convolutional filters whose outputs are transfomed by the ReLU activation function ($\text{ReLU}(x) = \max(0,x)$). On each of the 4 resulting vectors, we keep the maximal value ("max pooling") giving a us 4-long vector. We then perform a linear transformation of this 4-long vector into a single scalar ("Dense layer") which we map to the $[0,1]$ interval with sigmoid activation function $\text{sigm}(x) = \frac{1}{1+\exp(-x)} $ . The interpratation of the output is the probability that the sequence is bound by the transcription factor TAL1.

In [ ]:
model = Sequential()
###################################################################
# Fill your code here
# Use model.add() to add layers
# ConvDNA: cl.ConvDNA()
# GlobalMaxPooling1D: kl.GlobalMaxPooling1D()
# Dense: kl.Dense()
# Sigmoid: kl.Activation('sigmoid')
###################################################################

In [ ]:
from keras import optimizers

model.compile(optimizer=optimizers.Adam(lr=0.05), loss="binary_crossentropy")

from keras.callbacks import EarlyStopping

model.fit(x=x_train, y=y_train, epochs=300, verbose=2,
          callbacks=[
              EarlyStopping(patience=5)
          ],
          validation_split=.2
         )

In [ ]:
model.layers[0].plot_weights(figsize=(20, 12), plot_type="motif_pwm_info");

In [ ]:
y_test_pred = model.predict(x_test)

roc['ConvNet16Motif'] = roc_auc_score(y_test, y_test_pred)

In [ ]:
fpr['ConvNet16Motif'], tpr["ConvNet16Motif"], _ = roc_curve(y_test, y_test_pred)

In [ ]:
plt.plot(fpr['ConvNet16Motif'], tpr["ConvNet16Motif"], color='darkorange', 
        label = 'ConvNet16Motif ROC={%0.2f}' % roc['ConvNet16Motif'])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()